In [2]:
import os

os.chdir('D:/TTS_Trainer/')


In [11]:
import os
from os.path import exists, join, expanduser

!python -m pip install --upgrade pip

# Clone Deepvoice3_pytorch
name = "deepvoice3_pytorch" 
if not exists("./" + name):
  ! git clone https://github.com/r9y9/$name

os.chdir('D:/TTS_Trainer/deepvoice3_pytorch/')
!git checkout 7a10ac6763eda92595e257543494b6a95f64229b --quiet

os.chdir('D:/TTS_Trainer/')

# Install dependencices
%pip install --upgrade matplotlib torch lws synthesis train 
%pip install --upgrade hparams
%pip install -q "D:/TTS_Trainer/deepvoice3_pytorch/"
%pip install vs_buildtools tensorflow

%matplotlib inline
%pip install librosa nltk

import torch
import numpy as np
import librosa
import librosa.display
import IPython
from IPython.display import Audio
# need this for English text processing frontend
import nltk 
!python -m nltk.downloader cmudict

preset = "test/20180505_deepvoice3_ljspeech.json"
checkpoint_path = "test/checkpoint_1062000.pth"

# if not exists(preset):
#   !curl -O -L "https://www.dropbox.com/s/0ck82unm0bo0rxd/20180505_deepvoice3_ljspeech.json"
# if not exists(checkpoint_path):
#   !curl -O -L "https://www.dropbox.com/s/5ucl9remrwy5oeg/20180505_deepvoice3_checkpoint_step000640000.pth"

  Using cached synthesis-1.1061.tar.gz (629 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 22] Invalid argument: 'D:\\Ironman\\Temp\\pip-install-g9azrc5w\\synthesis_3b04ca8b00964977ba62dd50ff178582\\synthesis/failed_files/OCC_2010-12-08_21:14:31.668651_unencrypted.xml'



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [47 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'lws' extension
      creating build
      creating build\temp.win-amd64-cpython-38
      creating build\temp.win-amd64-cpython-38\Release
      creating build\temp.win-amd64-cpython-38\Release\lwslib
      "C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.29.30133\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -Ilwslib/ -Id:\TTS_Trainer\.conda\lib\site-packages\numpy\core\include -Id:\TTS_Trainer\.conda\include -Id:\TTS_Trainer\.conda\Include "-IC:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools\VC\Tools\MSVC\14.29.30133\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.19041.0\shared" "-IC:\Program Files (x86)\Windows Kits\

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement vs_buildtools (from versions: none)
ERROR: No matching distribution found for vs_buildtools


Note: you may need to restart the kernel to use updated packages.


d:\TTS_Trainer\.conda\lib\runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\ironman\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [13]:
import hparams
import json

    
# Load parameters from preset
with open(preset) as f:
  hparams.hparams.parse_json(f.read())
  
# Inject frontend text processor
import synthesis
import train
from deepvoice3_pytorch import frontend
synthesis._frontend = getattr(frontend, "en")
train._frontend =  getattr(frontend, "en")

# alises
fs = hparams.hparams.sample_rate
hop_length = hparams.hparams.hop_size

def tts(model, text, p=0, speaker_id=None, fast=True, figures=True):
  from synthesis import tts as _tts
  waveform, alignment, spectrogram, mel = _tts(model, text, p, speaker_id, fast)
  if figures:
      visualize(alignment, spectrogram)
  IPython.display.display(Audio(waveform, rate=fs))
  
def visualize(alignment, spectrogram):
  label_fontsize = 16
  figure(figsize=(16,16))

  subplot(2,1,1)
  imshow(alignment.T, aspect="auto", origin="lower", interpolation=None)
  xlabel("Decoder timestamp", fontsize=label_fontsize)
  ylabel("Encoder timestamp", fontsize=label_fontsize)
  colorbar()

  subplot(2,1,2)
  librosa.display.specshow(spectrogram.T, sr=fs, 
                           hop_length=hop_length, x_axis="time", y_axis="linear")
  xlabel("Time", fontsize=label_fontsize)
  ylabel("Hz", fontsize=label_fontsize)
  tight_layout()
  colorbar()

AttributeError: module 'hparams.hparams' has no attribute 'parse_json'

In [ ]:
from train import build_model
from train import restore_parts, load_checkpoint

model = build_model()
model = load_checkpoint(checkpoint_path, model, None, True)

In [ ]:
# Try your favorite senteneces:)
texts = [
    "Scientists at the CERN laboratory say they have discovered a new particle.",
    "There's a way to measure the acute emotional intelligence that has never gone out of style.",
    "President Trump met with other leaders at the Group of 20 conference.",
    "The Senate's bill to repeal and replace the Affordable Care Act is now imperiled.",
    "Generative adversarial network or variational auto-encoder.",
    "The buses aren't the problem, they actually provide a solution.",
    "peter piper picked a peck of pickled peppers how many peppers did peter piper pick.",
    "Some have accepted this as a miracle without any physical explanation.",
]

for idx, text in enumerate(texts):
  print(idx, text)
  tts(model, text, figures=False)

In [ ]:
# With attention plot
text = "Generative adversarial network or variational auto-encoder."
tts(model, text, figures=True)